In [14]:
import pandas as pd
import dask
import gev_bayes_utils as gevbu

from utils import city_list, gev_metric_ids
from utils import roar_data_path as project_data_path

In [7]:
############
### Dask ###
############
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    # account="pches",
    account="open",
    cores=1,
    memory="20GiB",
    walltime="00:30:00",
)

cluster.scale(jobs=20)  # ask for jobs

from dask.distributed import Client

client = Client(cluster)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.8.151:43229,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Get city timeseries

In [6]:
# Run for all
for city in city_list.keys():
    for metric_id in gev_metric_ids:
        gevbu.get_city_timeseries_all(city, metric_id)

## Fit Bayesian GEV

In [12]:
city = 'chicago'
metric_id = 'max_tasmax'
_, var_id = metric_id.split('_')
years = [2050,2100]

df = pd.read_csv(f'{project_data_path}/metrics/cities/{city}_{metric_id}.csv')
df = df.set_index(['ensemble', 'gcm', 'member', 'ssp']).sort_index()

In [19]:
%%time

stationary = True 

delayed = []

# Loop through all combos
for idx in df.index.unique():
    # Select GEV fit data
    df_sel = df.loc[idx]
    df_sel = df_sel[(df_sel['time'] >= years[0]) & (df_sel['time'] <= years[1])]
    
    # Skip invalid SSP-years combinations
    if len(df_sel) == 0:
        continue

    # Get data to fit
    data = df_sel[var_id].to_numpy()

    # Get info
    ensemble, gcm, member, ssp = idx
    info = city, metric_id, ensemble, gcm, member, ssp

    # Run
    df_tmp = dask.delayed(gevbu.fit_bayesian_gev_single)(
            data = data,
            info = info,
            loc_mu = 35,
            loc_sigma = 10,
            scale_sigma = 10,
            store_results = True,
            years = years,
            stationary = stationary
    )
    delayed.append(df_tmp)

df = pd.concat(dask.compute(*delayed))

AssertionError: Key not found in unpickled KeyData file. Verify the __eq__ and __hash__ functions of your Ops. The file is: /storage/home/dcl5300/.pytensor/compiledir_Linux-4.18-el8_10.x86_64-x86_64-with-glibc2.28-x86_64-3.12.7-64/tmp289trcvn/key.pkl. The key is: (((15, (5, (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4, 0), (4,), (4,)), (13, '1.26.4'), (13, '1.26.4'), (13, '1.26.4'), (13, '1.26.4'), (13, '1.26.4'), (13, '1.26.4'), (13, '1.26.4'), (13, '1.26.4'), (13, '1.26.4'), (13, '1.26.4'), ('openmp', False), ('openmp_elemwise_minsize', 200000)), ('scalar_op', 'inplace_pattern'), (11, 13, '1.26.4'), (11, 13, '1.26.4'), (11, 13, '1.26.4'), (11, 13, '1.26.4'), (11, 13, '1.26.4'), (11, 13, '1.26.4'), (11, 13, '1.26.4'), (11, 13, '1.26.4'), (11, 13, '1.26.4'), (11, 13, '1.26.4')), ('CLinker.cmodule_key', ('--param', '--param', '--param', '-DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION', '-O3', '-Wno-c++11-narrowing', '-Wno-unused-label', '-Wno-unuse

In [21]:
df

time     tasmax
ensemble  gcm        member   ssp                    
GARD-LENS CESM2-LENS 1001_01  ssp370  1950  34.675095
                              ssp370  1951  35.993076
                              ssp370  1952  35.384530
                              ssp370  1953  36.885980
                              ssp370  1954  35.872272
...                                    ...        ...
STAR-ESDM TaiESM1    r1i1p1f1 ssp585  2096  56.993553
                              ssp585  2097  62.858120
                              ssp585  2098  55.878680
                              ssp585  2099  64.782684
                              ssp585  2100  58.095530

[62171 rows x 2 columns]

In [ ]:
# Store combined
df.to_csv(f"{project_data_path}/extreme_value/cities/original_grid/combined/{city}_{metric_id}.csv", index=False)